# No. of Integer Bits = 1
# No. of Magnitude Bits = 12, 16

Here, we evaluate the accuracy of the Tanh activation function using the expression

tanh = sinh x / cosh x

Accuracies

**For 12 bits:** 96.6, 97.6, 97.7, 97.8, 97.8, 97.7

**For 16 bits:** 97.6, 97.7, 98.0, 97.9, 98.0, 98.0

In [ ]:
import multiprocessing 

import six
import numpy as np
import tensorflow.compat.v2 as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

!pip install git+https://github.com/google/qkeras

  Cloning https://github.com/google/qkeras to /tmp/pip-req-build-argn2ynw
  Running command git clone -q https://github.com/google/qkeras /tmp/pip-req-build-argn2ynw
     |████████████████████████████████| 174kB 7.5MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 6.8MB 12.0MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
  Created wheel for QKeras: filename=QKeras-0.8.0-cp36-none-any.whl size=143338 sha256=60a063bab87fded03f988541f4096af592a73be003e8ac6a4e90bdb344027f19
  Stored in directory: /tmp/pip-ephem-wheel-cache-spmqph4h/wheels/b4/74/1d/9456d62789716894a5edd7e342b4beaef69241ac584706c68d
  Created wheel for pyparser: filename=pyparser-1.0-cp36-none-any.whl size=4944 sha256=f155ff82f1d2e6667ff0edbd402bb43006b9008742450cad909e572e3c48deef
  Stored in directory: /root/.cache/pip/wheels/8a/1c/4f/9f66cd69719aa41c2684efae758d95db7803e9fe1f65146db1
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl

In [ ]:
def get_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape + (1,)).astype("float32")
    x_test = x_test.reshape(x_test.shape + (1,)).astype("float32")

    x_train /= 256.0
    x_test /= 256.0

    x_mean = np.mean(x_train, axis=0)

    x_train -= x_mean
    x_test -= x_mean

    nb_classes = np.max(y_train)+1
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = get_data()

In [ ]:
from qkeras import *
import qkeras
def CreateQModel(shape, nb_classes, intBits):
    x = x_in = Input(shape)

    x = Flatten(name="flatten")(x)
    x = QDense(256,
        kernel_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(13, {} , alpha = 1)".format(intBits),
        name="dense")(x)
    x = QActivation("quantized_tanh(13, {})".format(intBits), name="act_1")(x)
    x = QDense(128,
        kernel_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        name="dense2")(x)
    x = QActivation("quantized_tanh(13, {})".format(intBits), name="act_2")(x)
    x = QDense(128,
        kernel_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        name="dense3")(x)
    x = QActivation("quantized_tanh(13, {})".format(intBits), name="act_3")(x)
    x = QDense(nb_classes,
        kernel_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(13, {} , alpha=1)".format(intBits),
        name="dense4")(x)
    x = Activation("softmax", name="softmax")(x)


    model = Model(inputs=x_in, outputs=x)
    
    return model

In [ ]:
from qkeras.utils import *
from tensorflow.keras.optimizers import Adam
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=True)
qmodel = CreateQModel(x_train.shape[1:], y_train.shape[-1], 1)
qmodel.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(0.0005),
    metrics=["accuracy"])
history = qmodel.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test[:5000], y_test[:5000]), verbose=False, callbacks=[callback])
model_save_quantized_weights(qmodel)
print ("Done")

... quantizing model
Done


In [ ]:
qmodel.evaluate(x_test[5000:], y_test[5000:])

157/157 [==============================] - 0s 3ms/step - loss: 0.0678 - accuracy: 0.9808


[0.06781864911317825, 0.9807999730110168]

In [ ]:
tangents = 2**(-1*np.arange(1.0, 100.0, 1.0))
lookup_arctanh = np.arctanh(tangents)

def modifiedCordicTanh(arr, iterations):
  '''
  Returns the quantized tanh of the supplied argument x
  '''
  xarr = 1.2075*np.ones(shape=(len(arr), len(arr[0])))
  yarr = np.zeros(shape = (len(arr), len(arr[0])))

  global  lookup_arctanh
  #current_vector = np.array([1.2075, 0])
  #z = x

  for i in range(1, iterations+1):
    m = -1
    sigma = np.sign(arr)
    
    xchange = m*sigma*2**(-i)*yarr
    ychange = sigma*2**(-i)*xarr
    arrchange = sigma*lookup_arctanh[i-1]

    xarr -= xchange
    yarr += ychange
    arr -= arrchange
    
    #x = current_vector[0]
    #y = current_vector[1]
    #xnew = x
    #ynew = y
    #xnew = xnew - m*sigma*2**(-i)*y
    #ynew = ynew + sigma*2**(-i)*x
    #z = z - sigma*lookup_arctanh[i-1]
    #current_vector = [xnew, ynew]

  #ex = xarr + yarr
  #eminusx = xarr - yarr

  #temp1 = quantized_bits(9, 1, alpha=1)(ex - eminusx)
  #temp2 = quantized_bits(9, 1, alpha=1)(ex + eminusx)

  return quantized_bits(13,1, alpha=1)(yarr/xarr)

  #ex = current_vector[0] + current_vector[1]
  #eminusx = current_vector[0] - current_vector[1]

  #temp1 = quantized_bits(9,1,alpha=1)(ex - eminusx)
  #temp2 = quantized_bits(9, 1, alpha=1)(ex + eminusx)

  #return quantized_bits(9,1, alpha=1)(temp1/temp2)

In [ ]:
x_test = x_test[5000:]
y_test = y_test[5000:]

In [ ]:
def epicGeneratePredictions(indices, iterations):
  global justAHolder, numOfIterations
  from keras import backend as K
  get_third_layer_output = K.function([qmodel.layers[0].input],
                                    [qmodel.layers[2].output])
 
  layer3_output = get_third_layer_output([x_test])[0]

  layer3_output = layer3_output[indices[0]: indices[1]]

  layer3_output = modifiedCordicTanh(layer3_output, iterations)

  ### Let us try and sidestep this ###
  #for i in range(len(layer3_output)):
  #  for j in range(len(layer3_output[0])):
  #    layer3_output[i][j] = cordicSigmoid(layer3_output[i][j], iterations)

  #justAHolder = layer3_output
  #numOfIterations = iterations
  #arguments = np.dstack(np.meshgrid(np.array(range(len(layer3_output))), range(len(layer3_output[0])))).reshape(-1, 2)
  #doit(arguments)

  #layer3_output = justAHolder

  #print ("Done!!!")

  input_shape = qmodel.layers[4].get_input_shape_at(0)
  layer_input = Input(shape=(256))
  x = layer_input
  x = qmodel.layers[4](x)
  qm4 = Model(layer_input, x)

  predictions = qm4.predict(layer3_output)

  predictions = modifiedCordicTanh(predictions, iterations)

  #justAHolder = predictions
  #numOfIterations = iterations
  #arguments = np.dstack(np.meshgrid(np.array(range(len(predictions))), range(len(predictions[0])))).reshape(-1, 2)
  #doit(arguments)

  #predictions = justAHolder

  #for i in range(len(predictions)):
  #  for j in range(len(predictions[0])):
  #    predictions[i][j] = cordicSigmoid(predictions[i][j], iterations)

  input_shape = qmodel.layers[6].get_input_shape_at(0)
  layer_input = Input(shape=(128))
  x = layer_input
  x = qmodel.layers[6](x)
  qm4 = Model(layer_input, x)

  predictions = qm4.predict(predictions)


  predictions = modifiedCordicTanh(predictions, iterations)
  #justAHolder = predictions
  #numOfIterations = iterations
  #arguments = np.dstack(np.meshgrid(np.array(range(len(predictions))), range(len(predictions[0])))).reshape(-1, 2)
  #doit(arguments)

  #predictions = justAHolder

  #for i in range(len(predictions)):
  #  for j in range(len(predictions[0])):
  #    predictions[i][j] = cordicSigmoid(predictions[i][j], iterations)

  input_shape = qmodel.layers[8].get_input_shape_at(0)
  layer_input = Input(shape=(128))
  x = layer_input
  x = qmodel.layers[8](x)

  qm4 = Model(layer_input, x)
  predictions = np.array(qm4.predict(predictions))

  #answer = np.zeros_like(predictions)
  #answer[len(predictions), predictions.argmax()] = 1
  a = predictions
  return (a == a.max(axis=1)[:,None]).astype(int)
  #return answer

  #return answer




In [ ]:
answerVectors = []
for iter in [2,3,4, 5, 6, 7]:
  answerVectors.append(epicGeneratePredictions([0, 1000], iter))
  print ("Done")

Done
Done
Done
Done
Done
Done


In [ ]:
accuracy = []
for i in answerVectors:
  correct = 0
  for j in range(1000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

[966, 976, 977, 978, 978, 977]

# 16 Bits

In [ ]:
from qkeras import *
import qkeras
def CreateQModel(shape, nb_classes, intBits):
    x = x_in = Input(shape)

    x = Flatten(name="flatten")(x)
    x = QDense(256,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha = 1)".format(intBits),
        name="dense")(x)
    x = QActivation("quantized_tanh(17, {})".format(intBits), name="act_1")(x)
    x = QDense(128,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        name="dense2")(x)
    x = QActivation("quantized_tanh(17, {})".format(intBits), name="act_2")(x)
    x = QDense(128,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        name="dense3")(x)
    x = QActivation("quantized_tanh(17, {})".format(intBits), name="act_3")(x)
    x = QDense(nb_classes,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        name="dense4")(x)
    x = Activation("softmax", name="softmax")(x)


    model = Model(inputs=x_in, outputs=x)
    
    return model

In [ ]:
from qkeras.utils import *
from tensorflow.keras.optimizers import Adam
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=True)
qmodel = CreateQModel(x_train.shape[1:], y_train.shape[-1], 1)
qmodel.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(0.0005),
    metrics=["accuracy"])
history = qmodel.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test[:5000], y_test[:5000]), verbose=False, callbacks=[callback])
model_save_quantized_weights(qmodel)
print ("Done")

... quantizing model
Done


In [ ]:
tangents = 2**(-1*np.arange(1.0, 100.0, 1.0))
lookup_arctanh = np.arctanh(tangents)

def modifiedCordicTanh(arr, iterations):
  '''
  Returns the quantized tanh of the supplied argument x
  '''
  xarr = 1.2075*np.ones(shape=(len(arr), len(arr[0])))
  yarr = np.zeros(shape = (len(arr), len(arr[0])))

  global  lookup_arctanh
  #current_vector = np.array([1.2075, 0])
  #z = x

  for i in range(1, iterations+1):
    m = -1
    sigma = np.sign(arr)
    
    xchange = m*sigma*2**(-i)*yarr
    ychange = sigma*2**(-i)*xarr
    arrchange = sigma*lookup_arctanh[i-1]

    xarr -= xchange
    yarr += ychange
    arr -= arrchange
    
    #x = current_vector[0]
    #y = current_vector[1]
    #xnew = x
    #ynew = y
    #xnew = xnew - m*sigma*2**(-i)*y
    #ynew = ynew + sigma*2**(-i)*x
    #z = z - sigma*lookup_arctanh[i-1]
    #current_vector = [xnew, ynew]

  #ex = xarr + yarr
  #eminusx = xarr - yarr

  #temp1 = quantized_bits(9, 1, alpha=1)(ex - eminusx)
  #temp2 = quantized_bits(9, 1, alpha=1)(ex + eminusx)

  return quantized_bits(17,1, alpha=1)(yarr/xarr)

  #ex = current_vector[0] + current_vector[1]
  #eminusx = current_vector[0] - current_vector[1]

  #temp1 = quantized_bits(9,1,alpha=1)(ex - eminusx)
  #temp2 = quantized_bits(9, 1, alpha=1)(ex + eminusx)

  #return quantized_bits(9,1, alpha=1)(temp1/temp2)

In [ ]:
answerVectors = []
for iter in [2,3,4, 5, 6, 7]:
  answerVectors.append(epicGeneratePredictions([0, 1000], iter))
  print ("Done")

Done
Done
Done
Done
Done
Done


In [ ]:
accuracy = []
for i in answerVectors:
  correct = 0
  for j in range(1000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

[976, 977, 980, 979, 980, 980]